In [1]:
import sys
import os

print(sys.path)
# 現在のディレクトリをsys.pathに追加
sys.path.append(os.getcwd()+'code')
print(sys.path)

['/', '/usr/lib/python38.zip', '/usr/lib/python3.8', '/usr/lib/python3.8/lib-dynload', '', '/usr/local/lib/python3.8/dist-packages', '/usr/lib/python3/dist-packages']
['/', '/usr/lib/python38.zip', '/usr/lib/python3.8', '/usr/lib/python3.8/lib-dynload', '', '/usr/local/lib/python3.8/dist-packages', '/usr/lib/python3/dist-packages', '/code']


In [2]:
# area = '静岡県'
# station = '三島'
# amedas_list_path = '/code/amedas/'
# startmonth = '202403'
# endmonth = '202404'
# today = '2024-03-29'

# amedas = utils.Get_amedas_data(area_name=area, station_name=station, path=amedas_list_path)
# amedas.set_date1(startmonth=startmonth,endmonth=endmonth)
# amedas_data = amedas.dl_data('daily')

# # amedas_dataの整形
# amedas_data["year"] = amedas_data["年月"].astype(str).str[:4]
# amedas_data["month"] = amedas_data["年月"].astype(str).str[4:6]
# amedas_data["day"] = amedas_data["日"]
# amedas_data["date"] = pd.to_datetime(amedas_data[["year", "month", "day"]])
# amedas_data = amedas_data.set_index("date")

# # 2週間後の日付を取得
# pred_date = dt.strptime(today, '%Y-%m-%d') + datetime.timedelta(days=14)
# print(pred_date)
# amedas_data = amedas_data['2024-03-30':pred_date]

# # 必要な列のみ取得
# amedas_data = amedas_data[['平均気温', '最高気温', '最低気温']]
# amedas_data

## 2週間後予測

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Input, GRU, Dense
from keras.models import Model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import utils
import datetime
from datetime import datetime as dt

'''
データ読み込み
'''
today = '2024-04-20'
week_1 = 7
week_2 = 14
week_3 = 21
output_len = 50

# 手入力（temp_liは当日の値からなので、総数が予測分+1個となる）
week_number_li = [38,39,39,39,39,39,39,39,40,40,40,40,40,40,40,41,41,41,41,41,41]
temp_li = [18.2, 17.6, 18.2, 17.4, 17.0, 18.8, 19.0, 18.6, 19.2, 19.5, 20.0, 20.0, 19.5, 17.5, 19.8, 19.9, 22.1, 21.1, 18.0, 15.4, 16.2, 16.7]

pred_date = dt.strptime(today, '%Y-%m-%d') + datetime.timedelta(days=week_2)
df = pd.read_excel('/code/data/daily_data_ver3.xlsx')
df_calender = pd.read_csv('/code/data/calender.csv', index_col=0, parse_dates=True)
df = df[['date', 'Week Number', 'Week Start', 'Week End', '出荷量(ケース)', '定植本数', '平均気温', '最高気温', '最低気温']]
df = df[5:]

# 列名変更
df.rename(columns={'出荷量(ケース)': 'Shipment', '定植本数': 'Teisyoku', '平均気温': 'Temp', '最高気温': 'High Temp', '最低気温': 'Low Temp'}, inplace=True)
# print(df.head())

# 日付をインデックスに
df['date'] = pd.to_datetime(df['date'])
df.set_index('date', inplace=True)

# dfに14行追加
df_tail = pd.DataFrame({'Week Number': week_number_li[:-7], 'Shipment': [0]*week_2, 'Teisyoku': [0]*week_2, 'Temp': [0]*week_2, 'High Temp': [0]*week_2, 'Low Temp': [0]*week_2, 'Status': [0]*week_2, 'Week': [0]*week_2, 'Week Shipment': [0]*week_2, 'Week Teisyoku': [0]*week_2, 'Week Temp': [0]*week_2, 'Week High Temp': [0]*week_2, 'Week Low Temp': [0]*week_2, 'Week Status': [0]*week_2}, index=pd.date_range(start=df.index[-1] + pd.DateOffset(days=1), periods=week_2, freq='D'))
df = pd.concat([df, df_tail])

# Shipmentが0の日を0、それ以外を1とする列を作成
df['Status'] = df['Shipment'].apply(lambda x: 0 if x == 0 else 1)
# 未来の分のStatusを追加
df['Status'][today:] = df_calender['state'][today:pred_date.strftime('%Y-%m-%d')]

# 日曜日を1、月曜日を2、...、土曜日を7とする列を作成
df['Week'] = (df.index.weekday + 1) % 7 +1
df['Week'] = df['Week'].astype(int)
# print(df.head(10))

# 今日+1~14日後の気温を追加
df['Temp'][today:] = temp_li[:-7]

# 各列を週番号の1~7を足し合わせたリストを作る
Week_Shipment = []
Week_Teisyoku = []
Week_Temp = []
Week_High_Temp = []
Week_Low_Temp = []
Week_Status = []

# Weekが7になるまで足し合わせ、1になったらリセットするそれぞれの変数を作成
tmp_Shipment = 0  
tmp_Teisyoku = 0
tmp_Temp = 0
tmp_High_Temp = 0
tmp_Low_Temp = 0
tmp_Status = 0

for i in range(len(df)):
    tmp_Shipment += df['Shipment'][i]
    tmp_Teisyoku += df['Teisyoku'][i]
    tmp_Temp += df['Temp'][i]
    tmp_High_Temp += df['High Temp'][i]
    tmp_Low_Temp += df['Low Temp'][i]
    tmp_Status += df['Status'][i]
    if df['Week'][i] == 7:
        Week_Shipment.append(tmp_Shipment)
        Week_Teisyoku.append(tmp_Teisyoku)
        Week_Temp.append(tmp_Temp)
        Week_High_Temp.append(tmp_High_Temp)
        Week_Low_Temp.append(tmp_Low_Temp)
        Week_Status.append(tmp_Status)
        tmp_Shipment = 0
        tmp_Teisyoku = 0
        tmp_Temp = 0
        tmp_High_Temp = 0
        tmp_Low_Temp = 0
        tmp_Status = 0

# 最終行のWeekを格納
if df['Week'][i]==7:
    n = 0
else:
    n = df['Week'][i]

# 各リストの各値を7回ずつ繰り返す
Week_Shipment = np.array(Week_Shipment)
Week_Teisyoku = np.array(Week_Teisyoku)
Week_Temp = np.array(Week_Temp)
Week_High_Temp = np.array(Week_High_Temp)
Week_Low_Temp = np.array(Week_Low_Temp)
Week_Status = np.array(Week_Status)

Week_Shipment = np.repeat(Week_Shipment, 7)
Week_Teisyoku = np.repeat(Week_Teisyoku, 7)
Week_Temp = np.repeat(Week_Temp, 7)
Week_High_Temp = np.repeat(Week_High_Temp, 7)
Week_Low_Temp = np.repeat(Week_Low_Temp, 7)
Week_Status = np.repeat(Week_Status, 7)

# 週番号の端数分は週合計が計算できないため、0で埋めておく
Week_Shipment = np.append(Week_Shipment, np.zeros(n))
Week_Teisyoku = np.append(Week_Teisyoku, np.zeros(n))
Week_Temp = np.append(Week_Temp, np.zeros(n))
Week_High_Temp = np.append(Week_High_Temp, np.zeros(n))
Week_Low_Temp = np.append(Week_Low_Temp, np.zeros(n))

# 週番号の端数分は週合計が計算できないため、5で埋めておく
Week_Status = np.append(Week_Status, np.full(n, 5))

# dfに追加
df['Week Shipment'] = Week_Shipment
df['Week Teisyoku'] = Week_Teisyoku
df['Week Temp'] = Week_Temp
df['Week High Temp'] = Week_High_Temp
df['Week Low Temp'] = Week_Low_Temp
df['Week Status'] = Week_Status

df.tail(20)
# df.head(20)
# df_calender['2024-03-31':]

/tmp/ipykernel_3017318/1956665432.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Status'][today:] = df_calender['state'][today:pred_date.strftime('%Y-%m-%d')]
/tmp/ipykernel_3017318/1956665432.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Temp'][today:] = temp_li[:-7]


,Week Number,Week Start,Week End,Shipment,Teisyoku,Temp,High Temp,Low Temp,Status,Week,Week Shipment,Week Teisyoku,Week Temp,Week High Temp,Week Low Temp,Week Status
2024-04-15,38,2024-04-14,2024-04-20,1504.0,0,17.6,NaN,NaN,1,2,12998.0,0.0,127.2,NaN,NaN,6
2024-04-16,38,2024-04-14,2024-04-20,1884.0,0,18.2,NaN,NaN,1,3,12998.0,0.0,127.2,NaN,NaN,6
2024-04-17,38,2024-04-14,2024-04-20,2064.0,0,19.9,NaN,NaN,1,4,12998.0,0.0,127.2,NaN,NaN,6
2024-04-18,38,2024-04-14,2024-04-20,2015.0,0,18.5,NaN,NaN,1,5,12998.0,0.0,127.2,NaN,NaN,6
2024-04-19,38,2024-04-14,2024-04-20,2024.0,0,18.3,NaN,NaN,1,6,12998.0,0.0,127.2,NaN,NaN,6
2024-04-20,38,2024-04-14,2024-04-20,1830.0,0,18.2,NaN,NaN,0,7,12998.0,0.0,127.2,NaN,NaN,6
2024-04-21,38,NaT,NaT,0.0,0,17.6,0.0,0.0,1,1,0.0,0.0,126.6,0.0,0.0,5
2024-04-22,39,NaT,NaT,0.0,0,18.2,0.0,0.0,1,2,0.0,0.0,126.6,0.0,0.0,5
2024-04-23,39,NaT,NaT,0.0,0,17.4,0.0,0.0,0,3,0.0,0.0,126.6,0.0,0.0,5
2024-04-24,39,NaT,NaT,0.0,0,17.0,0.0,0.0,1,4,0.0,0.0,126.6,0.0,0.0,5


In [4]:
'''
データの前処理
'''
# Week ShipmentをWeek Statusで割った列を作成
df['Target2'] = df['Week Shipment'] / df['Week Status']
df['Target2'] = df['Target2'].shift(-week_2)
df['Target2'] = df['Target2'].fillna(0)

# Week Numberのそれぞれの値におけるWeek Shipmentの平均値を取得
Week_Shipment_Mean = df[:'2022-07-30'].groupby('Week Number')['Week Shipment'].mean().reset_index()
# Week Numberの値が等しい行にWeek Shipmentの平均値を結合
df['Week Shipment Mean'] = 0
for i in range(len(Week_Shipment_Mean)):
    df.loc[df['Week Number'] == Week_Shipment_Mean['Week Number'][i], 'Week Shipment Mean'] = Week_Shipment_Mean['Week Shipment'][i]

# Week Shipmentの値が0より大きい場合は1、それ以外は0のWeek Status2を作成
df['Week Status2'] = 0
df.loc[df['Week Shipment'] > 0, 'Week Status2'] = 1

# 既知のデータを2週間後のものに変更
df['Status'] = df['Status'].shift(-week_2)
df['Week Status'] = df['Week Status'].shift(-week_2)
df['Week Shipment Mean'] = df['Week Shipment Mean'].shift(-week_2)
df['Week Status2'] = df['Week Status2'].shift(-week_2)

# 環境データをshiftした列を作成
df['Temp' + str(-week_2)] = df['Temp'].shift(-week_2)
df['Week Temp' + str(-week_2)] = df['Week Temp'].shift(-week_2)

# Week Status2が0の場合は0とする
df.loc[df['Week Status2'] == 0, 'Temp-'+str(week_2)] = 0
df.loc[df['Week Status2'] == 0, 'Week Temp-'+str(week_2)] = 0
df.loc[df['Week Status2'] == 0, 'High Temp'] = 0
df.loc[df['Week Status2'] == 0, 'Week High Temp'] = 0
df.loc[df['Week Status2'] == 0, 'Low Temp'] = 0
df.loc[df['Week Status2'] == 0, 'Week Low Temp'] = 0

# 使用しない環境データを削除
df = df.drop(['Temp', 'Week Temp', 'High Temp', 'Week High Temp', 'Low Temp', 'Week Low Temp'], axis=1)

# target列を追加
df['Target'] = df['Week Shipment'].shift(-week_2)
# Week Shipment列を先週の結果とする
df['Week Shipment'] = df['Week Shipment'].shift(7)

# Week Numberをカテゴリ変数に変換
df['Week Number'] = df['Week Number'].astype('category')
df = pd.get_dummies(df, columns=['Week Number'], dtype=int)

# dataに必要な部分だけ抜き出す
# data = df[:-14]
data = df[-output_len-week_2:-week_2]
data.drop(['Target', 'Week', 'Week Start', 'Week End', 'Week Status2', 'Status', 'Teisyoku', 'Week Teisyoku'], axis=1, inplace=True)
print(data[['Temp-14', 'Week Temp-14']])

'''
モデル入力のための加工（正規化、次元サイズ整理）
'''
scaler = MinMaxScaler()

# featuresの各列を正規化
normalized_features = pd.DataFrame(scaler.fit_transform(data), columns=data.columns)

print(len(normalized_features))

# データセットの入力に合わせて列順の変更
features_df = pd.read_csv('/code/4月4日_GRUモデル/2week/GRU_features.csv', index_col=0)
feature_columns = features_df.columns
filtered_features = normalized_features[feature_columns]

# 平均値で欠損値を埋める
filtered_features.fillna(filtered_features.mean(), inplace=True)
# print(filtered_features[['Temp-14', 'Week Temp-14']])

# Xとyにデータを格納
X = filtered_features.drop('Target2', axis=1).values
# y = filtered_features['Target2'].values

# Xを3次元にリシェイプ
X = X.reshape((X.shape[0], X.shape[1], 1))

print(X.shape)

'''
モデルの推論
'''
# モデルの読み込み
from keras.models import load_model
current_model = load_model('/code/4月4日_GRUモデル/2week/gru_best_model_gru256_dense1024_loss0.0040.h5')

current_model.summary()

# 予測結果の計算
y_pred = current_model.predict(X)
print(len(y_pred))

# testデータの予測結果をDataFrameに格納
df = df.reset_index()
df.rename(columns={'index': 'date'}, inplace=True)
result2 = df[['date', 'Week Start', 'Week End']][-output_len-week_2:-week_2]
result2['pred_2week'] = y_pred[:, 0] * df['Target2'].max() * df['Week Status'][-output_len-week_2:-week_2].values
print(result2)
# result2.to_csv('/code/result_test.csv', index=False)

/tmp/ipykernel_3017318/772381309.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.drop(['Target', 'Week', 'Week Start', 'Week End', 'Week Status2', 'Status', 'Teisyoku', 'Week Teisyoku'], axis=1, inplace=True)
2024-04-22 01:06:18.973538: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-22 01:06:19.457071: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 21899 MB memory:  -> device: 0, name: NVIDIA RTX A5000, pci bus id: 0000:17:00.0, compute capa

            Temp-14  Week Temp-14
2024-03-02     12.5          68.2
2024-03-03     14.6          67.3
2024-03-04     12.3          67.3
2024-03-05      8.4          67.3
2024-03-06     11.1          67.3
2024-03-07      7.4          67.3
2024-03-08      6.5          67.3
2024-03-09      7.0          67.3
2024-03-10     10.9          93.8
2024-03-11     13.1          93.8
2024-03-12     10.9          93.8
2024-03-13     11.0          93.8
2024-03-14     12.3          93.8
2024-03-15     16.4          93.8
2024-03-16     19.2          93.8
2024-03-17     20.5         109.3
2024-03-18     17.0         109.3
2024-03-19     14.5         109.3
2024-03-20     13.9         109.3
2024-03-21     16.1         109.3
2024-03-22     12.2         109.3
2024-03-23     15.1         109.3
2024-03-24     17.6         110.1
2024-03-25     17.6         110.1
2024-03-26     17.0         110.1
2024-03-27     13.4         110.1
2024-03-28     13.4         110.1
2024-03-29     15.1         110.1
2024-03-30    

2024-04-22 01:06:20.888872: I tensorflow/stream_executor/cuda/cuda_blas.cc:1786] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


50
           date Week Start   Week End    pred_2week
2400 2024-03-02 2024-02-25 2024-03-02   9453.369751
2401 2024-03-03 2024-03-03 2024-03-09   9075.272827
2402 2024-03-04 2024-03-03 2024-03-09   9073.531494
2403 2024-03-05 2024-03-03 2024-03-09   9074.558716
2404 2024-03-06 2024-03-03 2024-03-09   9072.728271
2405 2024-03-07 2024-03-03 2024-03-09   9075.784302
2406 2024-03-08 2024-03-03 2024-03-09   9076.982422
2407 2024-03-09 2024-03-03 2024-03-09   9076.007690
2408 2024-03-10 2024-03-10 2024-03-15  11412.313232
2409 2024-03-11 2024-03-10 2024-03-15  11419.440918
2410 2024-03-12 2024-03-10 2024-03-15  11413.693848
2411 2024-03-13 2024-03-10 2024-03-15  11415.416260
2412 2024-03-14 2024-03-10 2024-03-15  11416.663818
2413 2024-03-15 2024-03-10 2024-03-15  11434.111328
2414 2024-03-16 2024-03-10 2024-03-15  11449.390869
2415 2024-03-17 2024-03-16 2024-03-23  15520.037354
2416 2024-03-18 2024-03-16 2024-03-23  15575.478516
2417 2024-03-19 2024-03-16 2024-03-23  15687.888672
2418 2024

2024-04-22 01:06:21.373786: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8101


In [5]:
df['Week Status'][-output_len-week_2:-week_2].values

array([5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 7., 7.,
       7., 7., 7., 7., 7., 7., 7., 7., 7., 7., 7., 7., 6., 6., 6., 6., 6.,
       6., 6., 5., 5., 5., 5., 5., 5., 5., 3., 3., 3., 3., 3., 3., 3.])

In [6]:
feature_columns

Index(['Target2', 'Week Shipment Mean', 'Week Shipment', 'Week Status',
       'Shipment', 'Week Number_38', 'Week Number_42', 'Week Number_39',
       'Week Number_43', 'Week Number_40', 'Week Number_41', 'Week Number_35',
       'Week Number_37', 'Week Number_44', 'Week Number_36', 'Week Number_45',
       'Week Temp-14', 'Temp-14', 'Week Number_46', 'Week Number_34',
       'Week Number_33', 'Week Number_21', 'Week Number_19', 'Week Number_20',
       'Week Number_22', 'Week Number_16', 'Week Number_17', 'Week Number_31',
       'Week Number_18', 'Week Number_30', 'Week Number_32', 'Week Number_15',
       'Week Number_25', 'Week Number_26', 'Week Number_29', 'Week Number_24',
       'Week Number_23', 'Week Number_27', 'Week Number_28', 'Week Number_47',
       'Week Number_13', 'Week Number_14', 'Week Number_12', 'Week Number_11',
       'Week Number_53', 'Week Number_48', 'Week Number_10', 'Week Number_49',
       'Week Number_9', 'Week Number_50', 'Week Number_51', 'Week Number_5

In [7]:
X[0]

array([[0.        ],
       [0.21717172],
       [0.5       ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.53616352],
       [0.6097561 ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [1.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.   

# 3週間後予測

In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Input, GRU, Dense
from keras.models import Model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import utils
import datetime
from datetime import datetime as dt

'''
データ読み込み
'''
pred_date = dt.strptime(today, '%Y-%m-%d') + datetime.timedelta(days=week_3)
df = pd.read_excel('/code/data/daily_data_ver3.xlsx')
df_calender = pd.read_csv('/code/data/calender.csv', index_col=0, parse_dates=True)
df = df[['date', 'Week Number', 'Week Start', 'Week End', '出荷量(ケース)', '定植本数', '平均気温', '最高気温', '最低気温']]
df = df[5:]

# 列名変更
df.rename(columns={'出荷量(ケース)': 'Shipment', '定植本数': 'Teisyoku', '平均気温': 'Temp', '最高気温': 'High Temp', '最低気温': 'Low Temp'}, inplace=True)
print(df.head())

# 日付をインデックスに
df['date'] = pd.to_datetime(df['date'])
df.set_index('date', inplace=True)

# dfに21行追加
df_tail = pd.DataFrame({'Week Number': week_number_li, 'Shipment': [0]*week_3, 'Teisyoku': [0]*week_3, 'Temp': [0]*week_3, 'High Temp': [0]*week_3, 'Low Temp': [0]*week_3, 'Status': [0]*week_3, 'Week': [0]*week_3, 'Week Shipment': [0]*week_3, 'Week Teisyoku': [0]*week_3, 'Week Temp': [0]*week_3, 'Week High Temp': [0]*week_3, 'Week Low Temp': [0]*week_3, 'Week Status': [0]*week_3}, index=pd.date_range(start=df.index[-1] + pd.DateOffset(days=1), periods=week_3, freq='D'))
df = pd.concat([df, df_tail])

# Shipmentが0の日を0、それ以外を1とする列を作成
df['Status'] = df['Shipment'].apply(lambda x: 0 if x == 0 else 1)
# 未来の分のStatusを追加
df['Status'][today:] = df_calender['state'][today:pred_date.strftime('%Y-%m-%d')]

# 日曜日を1、月曜日を2、...、土曜日を7とする列を作成
df['Week'] = (df.index.weekday + 1) % 7 +1
df['Week'] = df['Week'].astype(int)

# 今日+1~21日後の気温を追加(22日分)
df['Temp'][today:] = temp_li

# 各列を週番号の1~7を足し合わせたリストを作る
Week_Shipment = []
Week_Teisyoku = []
Week_Temp = []
Week_High_Temp = []
Week_Low_Temp = []
Week_Status = []

# Weekが7になるまで足し合わせ、1になったらリセットするそれぞれの変数を作成
tmp_Shipment = 0  
tmp_Teisyoku = 0
tmp_Temp = 0
tmp_High_Temp = 0
tmp_Low_Temp = 0
tmp_Status = 0

for i in range(len(df)):
    tmp_Shipment += df['Shipment'][i]
    tmp_Teisyoku += df['Teisyoku'][i]
    tmp_Temp += df['Temp'][i]
    tmp_High_Temp += df['High Temp'][i]
    tmp_Low_Temp += df['Low Temp'][i]
    tmp_Status += df['Status'][i]
    if df['Week'][i] == 7:
        Week_Shipment.append(tmp_Shipment)
        Week_Teisyoku.append(tmp_Teisyoku)
        Week_Temp.append(tmp_Temp)
        Week_High_Temp.append(tmp_High_Temp)
        Week_Low_Temp.append(tmp_Low_Temp)
        Week_Status.append(tmp_Status)
        tmp_Shipment = 0
        tmp_Teisyoku = 0
        tmp_Temp = 0
        tmp_High_Temp = 0
        tmp_Low_Temp = 0
        tmp_Status = 0

# 最終行のWeekを格納
if df['Week'][i]==7:
    n = 0
else:
    n = df['Week'][i]

# 各リストの各値を7回ずつ繰り返す
Week_Shipment = np.array(Week_Shipment)
Week_Teisyoku = np.array(Week_Teisyoku)
Week_Temp = np.array(Week_Temp)
Week_High_Temp = np.array(Week_High_Temp)
Week_Low_Temp = np.array(Week_Low_Temp)
Week_Status = np.array(Week_Status)

Week_Shipment = np.repeat(Week_Shipment, 7)
Week_Teisyoku = np.repeat(Week_Teisyoku, 7)
Week_Temp = np.repeat(Week_Temp, 7)
Week_High_Temp = np.repeat(Week_High_Temp, 7)
Week_Low_Temp = np.repeat(Week_Low_Temp, 7)
Week_Status = np.repeat(Week_Status, 7)

# 週番号の端数分は週合計が計算できないため、0で埋めておく
Week_Shipment = np.append(Week_Shipment, np.zeros(n))
Week_Teisyoku = np.append(Week_Teisyoku, np.zeros(n))
Week_Temp = np.append(Week_Temp, np.zeros(n))
Week_High_Temp = np.append(Week_High_Temp, np.zeros(n))
Week_Low_Temp = np.append(Week_Low_Temp, np.zeros(n))

# 週番号の端数分は週合計が計算できないため、5で埋めておく
Week_Status = np.append(Week_Status, np.full(n, 5))

# dfに追加
df['Week Shipment'] = Week_Shipment
df['Week Teisyoku'] = Week_Teisyoku
df['Week Temp'] = Week_Temp
df['Week High Temp'] = Week_High_Temp
df['Week Low Temp'] = Week_Low_Temp
df['Week Status'] = Week_Status

df.tail(20)
# df_calender['2024-03-31':]

        date  Week Number Week Start   Week End  Shipment  Teisyoku  Temp  \
5 2017-08-06            2 2017-08-06 2017-08-12       0.0         0  29.3   
6 2017-08-07            2 2017-08-06 2017-08-12       0.0         0  27.6   
7 2017-08-08            2 2017-08-06 2017-08-12       0.0      8648  28.3   
8 2017-08-09            2 2017-08-06 2017-08-12       0.0      8414  29.8   
9 2017-08-10            2 2017-08-06 2017-08-12       0.0         0  28.0   

   High Temp  Low Temp  
5       34.3      26.0  
6       33.2      25.4  
7       32.0      26.0  
8       35.6      24.0  
9       32.4      25.1  


/tmp/ipykernel_3017318/2048606404.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Status'][today:] = df_calender['state'][today:pred_date.strftime('%Y-%m-%d')]
/tmp/ipykernel_3017318/2048606404.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Temp'][today:] = temp_li


,Week Number,Week Start,Week End,Shipment,Teisyoku,Temp,High Temp,Low Temp,Status,Week,Week Shipment,Week Teisyoku,Week Temp,Week High Temp,Week Low Temp,Week Status
2024-04-22,39,NaT,NaT,0.0,0,18.2,0.0,0.0,1,2,0.0,0.0,126.6,0.0,0.0,5
2024-04-23,39,NaT,NaT,0.0,0,17.4,0.0,0.0,0,3,0.0,0.0,126.6,0.0,0.0,5
2024-04-24,39,NaT,NaT,0.0,0,17.0,0.0,0.0,1,4,0.0,0.0,126.6,0.0,0.0,5
2024-04-25,39,NaT,NaT,0.0,0,18.8,0.0,0.0,1,5,0.0,0.0,126.6,0.0,0.0,5
2024-04-26,39,NaT,NaT,0.0,0,19.0,0.0,0.0,1,6,0.0,0.0,126.6,0.0,0.0,5
2024-04-27,39,NaT,NaT,0.0,0,18.6,0.0,0.0,0,7,0.0,0.0,126.6,0.0,0.0,5
2024-04-28,39,NaT,NaT,0.0,0,19.2,0.0,0.0,0,1,0.0,0.0,135.5,0.0,0.0,3
2024-04-29,40,NaT,NaT,0.0,0,19.5,0.0,0.0,1,2,0.0,0.0,135.5,0.0,0.0,3
2024-04-30,40,NaT,NaT,0.0,0,20.0,0.0,0.0,1,3,0.0,0.0,135.5,0.0,0.0,3
2024-05-01,40,NaT,NaT,0.0,0,20.0,0.0,0.0,1,4,0.0,0.0,135.5,0.0,0.0,3


In [9]:
'''
データの前処理
'''
# Week ShipmentをWeek Statusで割った列を作成
df['Target2'] = df['Week Shipment'] / df['Week Status']
df['Target2'] = df['Target2'].shift(-week_3)
df['Target2'] = df['Target2'].fillna(0)

# Week Numberのそれぞれの値におけるWeek Shipmentの平均値を取得
Week_Shipment_Mean = df[:'2022-07-30'].groupby('Week Number')['Week Shipment'].mean().reset_index()
# Week Numberの値が等しい行にWeek Shipmentの平均値を結合
df['Week Shipment Mean'] = 0
for i in range(len(Week_Shipment_Mean)):
    df.loc[df['Week Number'] == Week_Shipment_Mean['Week Number'][i], 'Week Shipment Mean'] = Week_Shipment_Mean['Week Shipment'][i]

# Week Shipmentの値が0より大きい場合は1、それ以外は0のWeek Status2を作成
df['Week Status2'] = 0
df.loc[df['Week Shipment'] > 0, 'Week Status2'] = 1

# 既知のデータを3週間後のものに変更
df['Status'] = df['Status'].shift(-week_3)
df['Week Status'] = df['Week Status'].shift(-week_3)
df['Week Shipment Mean'] = df['Week Shipment Mean'].shift(-week_3)
df['Week Status2'] = df['Week Status2'].shift(-week_3)

# 環境データをshiftした列を作成
df['Temp' + str(-week_3)] = df['Temp'].shift(-week_3)
df['Week Temp' + str(-week_3)] = df['Week Temp'].shift(-week_3)

# Week Status2が0の場合は0とする
df.loc[df['Week Status2'] == 0, 'Temp-'+str(week_3)] = 0
df.loc[df['Week Status2'] == 0, 'Week Temp-'+str(week_3)] = 0
df.loc[df['Week Status2'] == 0, 'High Temp'] = 0
df.loc[df['Week Status2'] == 0, 'Week High Temp'] = 0
df.loc[df['Week Status2'] == 0, 'Low Temp'] = 0
df.loc[df['Week Status2'] == 0, 'Week Low Temp'] = 0

# 使用しない環境データを削除
df = df.drop(['Temp', 'Week Temp', 'High Temp', 'Week High Temp', 'Low Temp', 'Week Low Temp'], axis=1)

# target列を追加
df['Target'] = df['Week Shipment'].shift(-week_3)
# Week Shipment列を先週の結果とする
df['Week Shipment'] = df['Week Shipment'].shift(7)

# Week Numberをカテゴリ変数に変換
df['Week Number'] = df['Week Number'].astype('category')
df = pd.get_dummies(df, columns=['Week Number'], dtype=int)

# 推論に必要な部分だけ抜き出す
# data = df[:-14]
data = df[-output_len-week_3:-week_3]
data.drop(['Target', 'Week', 'Week Start', 'Week End', 'Week Status2', 'Status', 'Teisyoku', 'Week Teisyoku'], axis=1, inplace=True)


'''
モデル入力のための加工（正規化、次元サイズ整理）
'''
scaler = MinMaxScaler()

# featuresの各列を正規化
normalized_features = pd.DataFrame(scaler.fit_transform(data), columns=data.columns)

print(len(normalized_features))

# データセットの入力に合わせて列順の変更
features_df = pd.read_csv('/code/4月4日_GRUモデル/3week/GRU_features.csv', index_col=0)
feature_columns = features_df.columns
filtered_features = normalized_features[feature_columns]

# 平均値で欠損値を埋める
filtered_features.fillna(filtered_features.mean(), inplace=True)

# Xとyにデータを格納
X = filtered_features.drop('Target2', axis=1).values
# y = filtered_features['Target2'].values

# Xを3次元にリシェイプ
X = X.reshape((X.shape[0], X.shape[1], 1))

print(X.shape)

'''
モデルの推論
'''
# モデルの読み込み
from keras.models import load_model
current_model = load_model('/code/4月4日_GRUモデル/3week/gru_best_model_gru512_dense1024_loss0.0043.h5')

current_model.summary()

# 予測結果の計算
y_pred = current_model.predict(X)
print(len(y_pred))

# testデータの予測結果をDataFrameに格納
df = df.reset_index()
df.rename(columns={'index': 'date'}, inplace=True)
result3 = df[['date']][-output_len-week_3:-week_3]
result3['pred_3week'] = y_pred[:, 0] * df['Target2'].max() * df['Week Status'][-output_len-week_3:-week_3].values
print(result3)
# result3.to_csv('/code/result_test.csv', index=False)

50
(50, 59, 1)


/tmp/ipykernel_3017318/1451189168.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.drop(['Target', 'Week', 'Week Start', 'Week End', 'Week Status2', 'Status', 'Teisyoku', 'Week Teisyoku'], axis=1, inplace=True)


Model: "model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 59, 1)]           0         
                                                                 
 dense_10 (Dense)            (None, 59, 1024)          2048      
                                                                 
 gru_5 (GRU)                 (None, 512)               2362368   
                                                                 
 dense_11 (Dense)            (None, 1)                 513       
                                                                 
Total params: 2,364,929
Trainable params: 2,364,929
Non-trainable params: 0
_________________________________________________________________
50
           date    pred_3week
2400 2024-03-02   9055.300903
2401 2024-03-03  12186.589355
2402 2024-03-04  12245.971680
2403 2024-03-05  12245.629883
2404 2024-03-06  12147.93212

## 1週間後予測

In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Input, GRU, Dense
from keras.models import Model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import utils
import datetime
from datetime import datetime as dt

'''
データ読み込み
'''
pred_date = dt.strptime(today, '%Y-%m-%d') + datetime.timedelta(days=week_1)
df = pd.read_excel('/code/data/daily_data_ver3.xlsx')
df_calender = pd.read_csv('/code/data/calender.csv', index_col=0, parse_dates=True)
df = df[['date', 'Week Number', 'Week Start', 'Week End', '出荷量(ケース)', '定植本数', '平均気温', '最高気温', '最低気温']]
df = df[5:]

# 列名変更
df.rename(columns={'出荷量(ケース)': 'Shipment', '定植本数': 'Teisyoku', '平均気温': 'Temp', '最高気温': 'High Temp', '最低気温': 'Low Temp'}, inplace=True)
print(df.head())

# 日付をインデックスに
df['date'] = pd.to_datetime(df['date'])
df.set_index('date', inplace=True)

# dfに7行追加
df_tail = pd.DataFrame({'Week Number': week_number_li[:-14], 'Shipment': [0]*week_1, 'Teisyoku': [0]*week_1, 'Temp': [0]*week_1, 'High Temp': [0]*week_1, 'Low Temp': [0]*week_1, 'Status': [0]*week_1, 'Week': [0]*week_1, 'Week Shipment': [0]*week_1, 'Week Teisyoku': [0]*week_1, 'Week Temp': [0]*week_1, 'Week High Temp': [0]*week_1, 'Week Low Temp': [0]*week_1, 'Week Status': [0]*week_1}, index=pd.date_range(start=df.index[-1] + pd.DateOffset(days=1), periods=week_1, freq='D'))
df = pd.concat([df, df_tail])

# Shipmentが0の日を0、それ以外を1とする列を作成
df['Status'] = df['Shipment'].apply(lambda x: 0 if x == 0 else 1)
# 未来の分のStatusを追加
df['Status'][today:] = df_calender['state'][today:pred_date.strftime('%Y-%m-%d')]

# 日曜日を1、月曜日を2、...、土曜日を7とする列を作成
df['Week'] = (df.index.weekday + 1) % 7 +1
df['Week'] = df['Week'].astype(int)

# 今日+1~7日後の気温を追加(8日分)
df['Temp'][today:] = temp_li[:-14]

# 各列を週番号の1~7を足し合わせたリストを作る
Week_Shipment = []
Week_Teisyoku = []
Week_Temp = []
Week_High_Temp = []
Week_Low_Temp = []
Week_Status = []

# Weekが7になるまで足し合わせ、1になったらリセットするそれぞれの変数を作成
tmp_Shipment = 0  
tmp_Teisyoku = 0
tmp_Temp = 0
tmp_High_Temp = 0
tmp_Low_Temp = 0
tmp_Status = 0

for i in range(len(df)):
    tmp_Shipment += df['Shipment'][i]
    tmp_Teisyoku += df['Teisyoku'][i]
    tmp_Temp += df['Temp'][i]
    tmp_High_Temp += df['High Temp'][i]
    tmp_Low_Temp += df['Low Temp'][i]
    tmp_Status += df['Status'][i]
    if df['Week'][i] == 7:
        Week_Shipment.append(tmp_Shipment)
        Week_Teisyoku.append(tmp_Teisyoku)
        Week_Temp.append(tmp_Temp)
        Week_High_Temp.append(tmp_High_Temp)
        Week_Low_Temp.append(tmp_Low_Temp)
        Week_Status.append(tmp_Status)
        tmp_Shipment = 0
        tmp_Teisyoku = 0
        tmp_Temp = 0
        tmp_High_Temp = 0
        tmp_Low_Temp = 0
        tmp_Status = 0

# 最終行のWeekを格納
if df['Week'][i]==7:
    n = 0
else:
    n = df['Week'][i]

# 各リストの各値を7回ずつ繰り返す
Week_Shipment = np.array(Week_Shipment)
Week_Teisyoku = np.array(Week_Teisyoku)
Week_Temp = np.array(Week_Temp)
Week_High_Temp = np.array(Week_High_Temp)
Week_Low_Temp = np.array(Week_Low_Temp)
Week_Status = np.array(Week_Status)

Week_Shipment = np.repeat(Week_Shipment, 7)
Week_Teisyoku = np.repeat(Week_Teisyoku, 7)
Week_Temp = np.repeat(Week_Temp, 7)
Week_High_Temp = np.repeat(Week_High_Temp, 7)
Week_Low_Temp = np.repeat(Week_Low_Temp, 7)
Week_Status = np.repeat(Week_Status, 7)

# 週番号の端数分は週合計が計算できないため、0で埋めておく
Week_Shipment = np.append(Week_Shipment, np.zeros(n))
Week_Teisyoku = np.append(Week_Teisyoku, np.zeros(n))
Week_Temp = np.append(Week_Temp, np.zeros(n))
Week_High_Temp = np.append(Week_High_Temp, np.zeros(n))
Week_Low_Temp = np.append(Week_Low_Temp, np.zeros(n))

# 週番号の端数分は週合計が計算できないため、5で埋めておく
Week_Status = np.append(Week_Status, np.full(n, 5))

# dfに追加
df['Week Shipment'] = Week_Shipment
df['Week Teisyoku'] = Week_Teisyoku
df['Week Temp'] = Week_Temp
df['Week High Temp'] = Week_High_Temp
df['Week Low Temp'] = Week_Low_Temp
df['Week Status'] = Week_Status

df.tail(20)
# df_calender['2024-03-31':]

        date  Week Number Week Start   Week End  Shipment  Teisyoku  Temp  \
5 2017-08-06            2 2017-08-06 2017-08-12       0.0         0  29.3   
6 2017-08-07            2 2017-08-06 2017-08-12       0.0         0  27.6   
7 2017-08-08            2 2017-08-06 2017-08-12       0.0      8648  28.3   
8 2017-08-09            2 2017-08-06 2017-08-12       0.0      8414  29.8   
9 2017-08-10            2 2017-08-06 2017-08-12       0.0         0  28.0   

   High Temp  Low Temp  
5       34.3      26.0  
6       33.2      25.4  
7       32.0      26.0  
8       35.6      24.0  
9       32.4      25.1  


/tmp/ipykernel_3017318/2513373736.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Status'][today:] = df_calender['state'][today:pred_date.strftime('%Y-%m-%d')]
/tmp/ipykernel_3017318/2513373736.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Temp'][today:] = temp_li[:-14]


,Week Number,Week Start,Week End,Shipment,Teisyoku,Temp,High Temp,Low Temp,Status,Week,Week Shipment,Week Teisyoku,Week Temp,Week High Temp,Week Low Temp,Week Status
2024-04-08,37,2024-04-07,2024-04-13,1249.0,0,17.6,NaN,NaN,1,2,12428.0,0.0,110.1,NaN,NaN,7
2024-04-09,37,2024-04-07,2024-04-13,1874.0,0,17.0,NaN,NaN,1,3,12428.0,0.0,110.1,NaN,NaN,7
2024-04-10,37,2024-04-07,2024-04-13,1995.0,0,13.4,NaN,NaN,1,4,12428.0,0.0,110.1,NaN,NaN,7
2024-04-11,37,2024-04-07,2024-04-13,2071.0,0,13.4,NaN,NaN,1,5,12428.0,0.0,110.1,NaN,NaN,7
2024-04-12,37,2024-04-07,2024-04-13,2048.0,0,15.1,NaN,NaN,1,6,12428.0,0.0,110.1,NaN,NaN,7
2024-04-13,37,2024-04-07,2024-04-13,1923.0,0,16.0,NaN,NaN,1,7,12428.0,0.0,110.1,NaN,NaN,7
2024-04-14,38,2024-04-14,2024-04-20,1677.0,0,16.5,NaN,NaN,1,1,12998.0,0.0,127.2,NaN,NaN,6
2024-04-15,38,2024-04-14,2024-04-20,1504.0,0,17.6,NaN,NaN,1,2,12998.0,0.0,127.2,NaN,NaN,6
2024-04-16,38,2024-04-14,2024-04-20,1884.0,0,18.2,NaN,NaN,1,3,12998.0,0.0,127.2,NaN,NaN,6
2024-04-17,38,2024-04-14,2024-04-20,2064.0,0,19.9,NaN,NaN,1,4,12998.0,0.0,127.2,NaN,NaN,6


In [11]:
'''
データの前処理
'''
# Week ShipmentをWeek Statusで割った列を作成
df['Target2'] = df['Week Shipment'] / df['Week Status']
df['Target2'] = df['Target2'].shift(-week_1)
df['Target2'] = df['Target2'].fillna(0)

# Week Numberのそれぞれの値におけるWeek Shipmentの平均値を取得
Week_Shipment_Mean = df[:'2022-07-30'].groupby('Week Number')['Week Shipment'].mean().reset_index()
# Week Numberの値が等しい行にWeek Shipmentの平均値を結合
df['Week Shipment Mean'] = 0
for i in range(len(Week_Shipment_Mean)):
    df.loc[df['Week Number'] == Week_Shipment_Mean['Week Number'][i], 'Week Shipment Mean'] = Week_Shipment_Mean['Week Shipment'][i]

# Week Shipmentの値が0より大きい場合は1、それ以外は0のWeek Status2を作成
df['Week Status2'] = 0
df.loc[df['Week Shipment'] > 0, 'Week Status2'] = 1

# 既知のデータを1週間後のものに変更
df['Status'] = df['Status'].shift(-week_1)
df['Week Status'] = df['Week Status'].shift(-week_1)
df['Week Shipment Mean'] = df['Week Shipment Mean'].shift(-week_1)
df['Week Status2'] = df['Week Status2'].shift(-week_1)

# 環境データをshiftした列を作成
df['Temp' + str(-week_1)] = df['Temp'].shift(-week_1)
df['Week Temp' + str(-week_1)] = df['Week Temp'].shift(-week_1)

# Week Status2が0の場合は0とする
df.loc[df['Week Status2'] == 0, 'Temp-'+str(week_1)] = 0
df.loc[df['Week Status2'] == 0, 'Week Temp-'+str(week_1)] = 0
df.loc[df['Week Status2'] == 0, 'High Temp'] = 0
df.loc[df['Week Status2'] == 0, 'Week High Temp'] = 0
df.loc[df['Week Status2'] == 0, 'Low Temp'] = 0
df.loc[df['Week Status2'] == 0, 'Week Low Temp'] = 0

# 使用しない環境データを削除
df = df.drop(['Temp', 'Week Temp', 'High Temp', 'Week High Temp', 'Low Temp', 'Week Low Temp'], axis=1)

# target列を追加
df['Target'] = df['Week Shipment'].shift(-week_1)
# Week Shipment列を先週の結果とする
df['Week Shipment'] = df['Week Shipment'].shift(7)

# Week Numberをカテゴリ変数に変換
# df['Week Number'] = df['Week Number'].astype('category')
# df = pd.get_dummies(df, columns=['Week Number'], dtype=int)

data = df

# Week Numberをカテゴリ変数に変換
data['Week Number'] = data['Week Number'].astype('category')
data = pd.get_dummies(data, columns=['Week Number'], dtype=int)

# 推論に必要な部分だけ抜き出す
data = data[-output_len-week_1:-week_1]
data.drop(['Target', 'Week', 'Week Start', 'Week End', 'Week Status2', 'Status', 'Teisyoku', 'Week Teisyoku'], axis=1, inplace=True)


'''
モデル入力のための加工（正規化、次元サイズ整理）
'''
scaler = MinMaxScaler()

# featuresの各列を正規化
normalized_features = pd.DataFrame(scaler.fit_transform(data), columns=data.columns)

print(len(normalized_features))

# データセットの入力に合わせて列順の変更
features_df = pd.read_csv('/code/4月4日_GRUモデル/1week/GRU_features.csv', index_col=0)
feature_columns = features_df.columns
filtered_features = normalized_features[feature_columns]

# 平均値で欠損値を埋める
filtered_features.fillna(filtered_features.mean(), inplace=True)

# Xとyにデータを格納
X = filtered_features.drop('Target2', axis=1).values
# y = filtered_features['Target2'].values

# Xを3次元にリシェイプ
X = X.reshape((X.shape[0], X.shape[1], 1))

print(X.shape)

'''
モデルの推論
'''
# モデルの読み込み
from keras.models import load_model
current_model = load_model('/code/4月4日_GRUモデル/1week/gru_best_model_gru512_dense1024_loss0.0045.h5')

current_model.summary()

# 予測結果の計算
y_pred = current_model.predict(X)
print(len(y_pred))

# testデータの予測結果をDataFrameに格納
df = df.reset_index()
df.rename(columns={'index': 'date'}, inplace=True)
result1 = df[['date', 'Week Start', 'Week End', 'Week Number']][-output_len-week_1:-week_1]
result1['pred_1week'] = y_pred[:, 0] * df['Target2'].max() * df['Week Status'][-output_len-week_1:-week_1].values
print(result1)
# result1.to_csv('/code/result_test.csv', index=False)

50
(50, 59, 1)


Model: "model_41"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_42 (InputLayer)       [(None, 59, 1)]           0         
                                                                 
 dense_82 (Dense)            (None, 59, 1024)          2048      
                                                                 
 gru_41 (GRU)                (None, 512)               2362368   
                                                                 
 dense_83 (Dense)            (None, 1)                 513       
                                                                 
Total params: 2,364,929
Trainable params: 2,364,929
Non-trainable params: 0
_________________________________________________________________
50
           date Week Start   Week End Week Number    pred_1week
2400 2024-03-02 2024-02-25 2024-03-02          31   9307.066650
2401 2024-03-03 2024-03-03 2024-03-09          32

## JAフォーマットへ変換するためのcsvを出力

In [12]:
result1['pred_2week'] = result2['pred_2week']
result1['pred_3week'] = result3['pred_3week']
result1

,date,Week Start,Week End,Week Number,pred_1week,pred_2week,pred_3week
2400,2024-03-02,2024-02-25,2024-03-02,31,9307.066650,9453.369751,9055.300903
2401,2024-03-03,2024-03-03,2024-03-09,32,9585.803223,9075.272827,12186.589355
2402,2024-03-04,2024-03-03,2024-03-09,32,9589.139404,9073.531494,12245.971680
2403,2024-03-05,2024-03-03,2024-03-09,32,9599.644775,9074.558716,12245.629883
2404,2024-03-06,2024-03-03,2024-03-09,32,9590.051880,9072.728271,12147.932129
2405,2024-03-07,2024-03-03,2024-03-09,32,9591.015625,9075.784302,12212.648926
2406,2024-03-08,2024-03-03,2024-03-09,32,9592.766113,9076.982422,12365.698242
2407,2024-03-09,2024-03-03,2024-03-09,32,9607.934570,9076.007690,12510.543213
2408,2024-03-10,2024-03-10,2024-03-15,33,9184.628296,11412.313232,16559.525391
2409,2024-03-11,2024-03-10,2024-03-15,33,9180.929565,11419.440918,16594.675781


In [13]:
result1.to_csv('/code/result_test.csv')

## グラフ表示用のcsv作成

In [14]:
# データフレームの作成
num = 6 # 実績値の数
date = result1['Week Start'].iloc[-num*7::7].reset_index(drop=True)
this_year = df['Week Shipment'].iloc[-num*6+n::7].reset_index(drop=True)
print(this_year)
week_number = result1['Week Number'].iloc[-num*7::7].reset_index(drop=True)

graph_df = pd.DataFrame({'Week Start': date, 'Week Number': week_number, '今年': this_year, '去年': [0]*num})
graph_df

0     9371.0
1     8644.0
2     8783.0
3    10662.0
4    12428.0
5    12998.0
Name: Week Shipment, dtype: float64


,Week Start,Week Number,今年,去年
0,2024-03-10,33,9371.0,0
1,2024-03-16,34,8644.0,0
2,2024-03-24,35,8783.0,0
3,2024-03-30,36,10662.0,0
4,2024-04-07,37,12428.0,0
5,2024-04-14,38,12998.0,0


In [15]:
# 当週のデータを入力
graph_df['今年'].iloc[-1] = result1['pred_1week'].iloc[-2]

/tmp/ipykernel_3017318/1285427891.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  graph_df['今年'].iloc[-1] = result1['pred_1week'].iloc[-2]


In [16]:
# 予測値のデータフレーム作成
date = [graph_df['Week Start'].iloc[-1] + datetime.timedelta(days=7*i) for i in range(1, 4)]
this_year = [result1['pred_1week'].iloc[-1], result2['pred_2week'].iloc[-1], result3['pred_3week'].iloc[-1]]
week_number = [graph_df['Week Number'].iloc[-1] + i for i in range(1, 4)]

pred_df = pd.DataFrame({'Week Start': date, 'Week Number': week_number, '今年': this_year, '去年': [0]*3})
pred_df

,Week Start,Week Number,今年,去年
0,2024-04-21,39,15186.486816,0
1,2024-04-28,40,11084.894531,0
2,2024-05-05,41,13494.884766,0


In [17]:
# データフレームの結合
graph_df = pd.concat([graph_df, pred_df], axis=0)
graph_df = graph_df.reset_index(drop=True)
graph_df

,Week Start,Week Number,今年,去年
0,2024-03-10,33,9371.000000,0
1,2024-03-16,34,8644.000000,0
2,2024-03-24,35,8783.000000,0
3,2024-03-30,36,10662.000000,0
4,2024-04-07,37,12428.000000,0
5,2024-04-14,38,15220.908203,0
6,2024-04-21,39,15186.486816,0
7,2024-04-28,40,11084.894531,0
8,2024-05-05,41,13494.884766,0


In [18]:
# 去年のデータを取得
LastYear_Index = []
for i in range(len(graph_df)):
    df_bool = (graph_df["Week Start"].dt.year.iloc[i]-1 == df["date"].dt.year) & (df["Week Number"] == graph_df["Week Number"].iloc[i])
    print(df_bool[df_bool==True].index[0])
    LastYear_Index.append(df_bool[df_bool==True].index[0])
print(LastYear_Index)

graph_df["去年"] = df["Week Shipment"].shift(-7).iloc[LastYear_Index].values # 特徴量計算時に7日前のデータを使用しているため、7日シフト
graph_df

2044
2051
2058
2065
2072
2079
2086
2093
2100
[2044, 2051, 2058, 2065, 2072, 2079, 2086, 2093, 2100]


,Week Start,Week Number,今年,去年
0,2024-03-10,33,9371.000000,9333.000000
1,2024-03-16,34,8644.000000,9920.999999
2,2024-03-24,35,8783.000000,10246.000000
3,2024-03-30,36,10662.000000,11775.000000
4,2024-04-07,37,12428.000000,13053.000000
5,2024-04-14,38,15220.908203,12136.000000
6,2024-04-21,39,15186.486816,12798.000000
7,2024-04-28,40,11084.894531,15022.000000
8,2024-05-05,41,13494.884766,15496.000000


In [19]:
graph_df.drop("Week Number", axis=1, inplace=True)
graph_df

,Week Start,今年,去年
0,2024-03-10,9371.000000,9333.000000
1,2024-03-16,8644.000000,9920.999999
2,2024-03-24,8783.000000,10246.000000
3,2024-03-30,10662.000000,11775.000000
4,2024-04-07,12428.000000,13053.000000
5,2024-04-14,15220.908203,12136.000000
6,2024-04-21,15186.486816,12798.000000
7,2024-04-28,11084.894531,15022.000000
8,2024-05-05,13494.884766,15496.000000


In [20]:
graph_df.to_csv(f'/code/4月4日_GRUモデル/pred_GRU_{today}_ver2.csv', index=False)